In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Загрузка данных
train = pd.read_csv('/content/train_data.csv')
test = pd.read_csv('/content/test_data.csv')
sam_sub = pd.read_csv('/content/sample_submission_data.csv')

In [ ]:
# Предобработка данных
categorical_cols = train.select_dtypes(include=['object']).columns.intersection(test.columns)
for col in categorical_cols:
    train[col].fillna(train[col].mode()[0], inplace=True)
    test[col].fillna(test[col].mode()[0], inplace=True)

In [ ]:
# Заполнение пропусков для числовых колонок
numeric_cols = [col for col in train.select_dtypes(include=['float64', 'int64']).columns if col in test.columns]

train[numeric_cols] = train[numeric_cols].fillna(train[numeric_cols].mean())
test[numeric_cols] = test[numeric_cols].fillna(test[numeric_cols].mean())

In [ ]:
test.isnull().sum()

,0
id,0
Name,0
Gender,0
Age,0
City,0
Working Professional or Student,0
Profession,0
Academic Pressure,0
Work Pressure,0
CGPA,0


In [ ]:
train.isnull().sum()

,0
id,0
Name,0
Gender,0
Age,0
City,0
Working Professional or Student,0
Profession,0
Academic Pressure,0
Work Pressure,0
CGPA,0


In [ ]:
# Кодирование категориальных переменных
train = pd.get_dummies(train, drop_first=True)
test = pd.get_dummies(test, drop_first=True)
# Убедимся, что тестовый набор имеет те же колонки, что и тренировочный
test = test.reindex(columns=train.columns, fill_value=0)
# Определение признаков и целевой переменной
X = train.drop('Depression', axis=1)
y = train['Depression']

In [ ]:
# Разделение данных на тренировочную и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

🤖 Model Building (using DummyClassifier and XGBoost)


In [ ]:
# 1. Измерение качества константного предсказания
most_frequent_class = y_train.mode()[0]
y_pred_constant = [most_frequent_class] * len(y_val)
accuracy_constant = accuracy_score(y_val, y_pred_constant)
print(f'Accuracy of constant prediction: {accuracy_constant}')

Accuracy of constant prediction: 0.8168443496801706


**Вывод:** Точность 0.8168443496801706% указывает на то, что модель предсказывает наиболее частый класс для всех примеров в валидационной выборке.
Этот результат служит базовым уровнем для оценки других моделей, так как он показывает, что даже простая стратегия может достичь значительной точности.    

In [ ]:
# 2. Бейзлайновая модель с DummyClassifier
# Я выбрала DummyClassifier, так как он позволяет установить базовый уровень точности для задачи классификации. Использовала стратегию 'most_frequent', чтобы предсказать наиболее частый класс.
dummy_model = DummyClassifier(strategy='most_frequent', random_state=42)
dummy_model.fit(X_train, y_train)
y_pred_dummy = dummy_model.predict(X_val)

In [ ]:
# 3. Оценка качества DummyClassifier
accuracy_dummy = accuracy_score(y_val, y_pred_dummy)
print(f'Accuracy of DummyClassifier: {accuracy_dummy}')
print("\nClassification Report for DummyClassifier:\n", classification_report(y_val, y_pred_dummy))

Accuracy of DummyClassifier: 0.8168443496801706

Classification Report for DummyClassifier:
               precision    recall  f1-score   support

           0       0.82      1.00      0.90     22986
           1       0.00      0.00      0.00      5154

    accuracy                           0.82     28140
   macro avg       0.41      0.50      0.45     28140
weighted avg       0.67      0.82      0.73     28140



**Вывод:** Результат по точности оказался идентичным 0.8168443496801706%. Однако, при более детальном анализе, представленный классификационный отчет показал, что модель имеет серьезные недостатки в предсказании класса 1.
Precision для класса 1 равен 0.00, что означает, что модель не предсказала ни одного положительного примера. Recall для класса 1 также равен 0.00, что указывает на то, что модель не смогла обнаружить ни одного истинного положительного примера. Это приводит к тому, что F1-score для класса 1 равен 0.00, что является критическим недостатком, особенно если класс 1 важен для вашей задачи. Это подчеркивает необходимость использования более сложных моделей и методов, которые могут лучше справляться с дисбалансом классов

In [ ]:
# 4. Обучение XGBClassifier (продвинутая модель)
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

In [ ]:
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [ ]:
y_pred_xgb = xgb_model.predict(X_val)

In [ ]:
print("Accuracy:", accuracy_score(y_val, y_pred_xgb))
print("\nClassification Report:\n", classification_report(y_val, y_pred_xgb))

Accuracy: 0.9381307746979389

Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.96      0.96     22986
           1       0.84      0.82      0.83      5154

    accuracy                           0.94     28140
   macro avg       0.90      0.89      0.90     28140
weighted avg       0.94      0.94      0.94     28140



**Вывод:** Для улучшения точности я использовала XGBClassifier, который является мощным инструментом для задач классификации, основанным на градиентном бустинге. Это позволяет модели учитывать сложные зависимости в данных и улучшать качество предсказаний. После обучения модели вы сделали предсказания на валидационной выборке и получили точность 0.9381307746979389
. Это значительно выше, чем результаты, полученные с помощью константного предсказания и DummyClassifier, что указывает на улучшение качества модели. Precision для класса 0 составил 0.96, что означает, что модель очень точно предсказывает отрицательные примеры. Recall для класса 0 также равен 0.96, что указывает на высокую способность модели обнаруживать все истинные отрицательные примеры. Для класса 1, Precision составил 0.84, а Recall — 0.82. Это говорит о том, что модель также хорошо справляется с предсказанием положительных примеров, хотя и с некоторыми ошибками. F1-score для класса 1 равен 0.83, что является хорошим показателем, учитывая, что этот класс был менее представлен в данных. Результаты показывают, что использование более сложных моделей, таких как XGBClassifier, может значительно улучшить качество предсказаний в задачах классификации, особенно когда данные имеют сложные зависимости.